In [8]:
import h5py
import os
import os.path as op

filename = 'test.hdf5'

if op.exists(filename):
    os.remove(filename)
    
f = h5py.File(filename, 'w')
hum_size = 3137161264
#hum_size = 8000000

'''
max_zoom = 25
dsets = []
for i in range(max_zoom):
    dsets += [f.create_dataset("zipped_{}".format(i), (hum_size / 2 ** i,), dtype='f', compression="gzip")]
'''

'\nmax_zoom = 25\ndsets = []\nfor i in range(max_zoom):\n    dsets += [f.create_dataset("zipped_{}".format(i), (hum_size / 2 ** i,), dtype=\'f\', compression="gzip")]\n'

### Encoding file values using a lookup table

The majority of values in a BED-like file are not unique. If we can store them in a lookup table, then we can save space by using an `int16` rather than a `float32` to represent them.

In [9]:
%%bash
# take a subset of a file so we don't have to wait for the whole file to be loaded
#zcat ~/data/encode/hg19/E002-H3K27me3.fc.signal.bigwig.genome.sorted.gz | head -n 1000000000 | gzip > /tmp/bgcompx1
zcat E044-H3K27me3.fc.signal.bigwig.genome.sorted.gz | head -n 10000 | gzip > /tmp/bgcompx1


In [3]:
import gzip
import numpy as np
import os
import os.path as op
import time as time

# 
# where our subset resides
filepath='/tmp/bgcompx1'

In [4]:
import csv
import h5py

t1 = time.time()
hum_size = 3137161264


with gzip.open(filepath, 'rt') as f:
    # create an empty buffer the size of the human genome
    reduced_buffer = np.zeros(hum_size, dtype='int16')
    
    lookup_table = {}
    
    for n,line in enumerate(csv.reader(f, delimiter='\t')):
        parts = line
        
        if parts[0] != 0:
            # we ignore values with an entry of 0 because our array is already initialized to 0
            num = float(parts[2])
            
            if num not in lookup_table:
                lookup_table[num] = len(lookup_table)
            
            reduced_buffer[int(parts[0])-1:int(parts[1])-1] = lookup_table[num]
                
        '''
        for i in range(int(parts[0]), int(parts[1])):
            buffer1 += [float(parts[2]) - prev_value]
            prev_value = float(parts[2])
        '''
print("time to read:", time.time() - t1)


time to read: 0.025804758071899414


In [5]:
print("len(lookup_table)", len(lookup_table))

len(lookup_table) 1060


In [6]:
filename = 'test1.hdf5'
if op.exists(filename):
    os.remove(filename)
f = h5py.File(filename, 'w')
hum_size = 3137161264

if len(lookup_table) < 2 ** 16:
    print("Creating 16bit data")
    dset = f.create_dataset('bedgraph', (hum_size,), dtype=np.int16, compression='gzip')
else:
    print("Creating 32bit data")
    dset = f.create_dataset('bedgraph', (hum_size,), dtype=np.int32, compression='gzip')

dset[0:len(reduced_buffer)] = reduced_buffer

f.flush()
f.close()

Creating 16bit data


In [7]:
print("file_size:", op.getsize(filename))
print("original file size:", op.getsize(filepath))

file_size: 8069484
original file size: 62860
